## RNN의 입출력 단위가 단어 레벨(word-level)이 아니라 문자 레벨(character-level)로 하여 RNN을 구현한다면, 이를 문자 단위 RNN이라고 한다. 다대다 구조로 구현해보자. 

### 1. 문자 단위 RNN(Char RNN)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

#### 1) 훈련 데이터 전처리하기

입력 데이터와 레이블 데이터에 대해서 문자 집합(vocabulary)를 만든다.  
여기서 문자 집합은 중복을 제거한 문자들의 집합이다.  

In [2]:
input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str+label_str)))
vocab_size = len(char_vocab)
print(f'문자 집합의 크기 : {vocab_size}')

문자 집합의 크기 : 5


문자 집합의 문자는 총 5개이다. '!,a,e,l,p'.

이제 하이퍼파라미터를 정의한다.  
입력은 원-핫 벡터를 사용할 것이므로 입력의 크기는 문자 집합의 크기여야만 한다.

In [3]:
input_size = vocab_size # 입력의 크기는 문자 집합의 크기
hidden_size = 5
output_size = 5
learning_rate = 0.1

문자 집합에 고유한 정수를 부여한다. 

In [4]:
char_to_index = dict((c,i) for i,c in enumerate(char_vocab))
# 문자에 고유한 정수 인덱스 부여
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


나중에 예측 결과를 다시 문자 시퀀스로 보기 위해서   
반대로 정수로부터 문자를 얻을 수 있는 index_to_char를 만든다.

In [5]:
index_to_char = {}
for key, value in char_to_index.items():
    index_to_char[value] = key

print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


이제 입력 데이터와 레이블 데이터의 각 문자들을 정수로 맵핑한다.

In [6]:
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data)
print(y_data)

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


파이토치의 nn.RNN()은 기본적으로 3차원 텐서를 입력받는다.  
따라서 배치 차원을 추가해준다.

In [7]:
# 배치 차원 추가
# 텐서 연산인 unsqueeze(0)를 통해 해결할 수도 있음.
x_data = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


입력 시퀀스의 각 문자들을 원-핫 벡터로 바꿔준다.

In [8]:
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


입력 데이터와 레이블 데이터를 텐서로 바꿔준다.

In [9]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

C:\Users\user\AppData\Local\Temp\ipykernel_17804\2348034151.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  X = torch.FloatTensor(x_one_hot)


이제 각 텐서의 크기를 확인해보자.

In [10]:
print(f'훈련 데이터의 크기 : {X.shape}')
print(f'레이블의 크기 : {Y.shape}')

훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([1, 5])


#### 2) 모델 구현하기

이제 RNN 모델을 구현하자.  
fc는 fully-connected layer를 뜻하며, 출력층으로 사용된다.

In [11]:
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_size,hidden_size,batch_first=True) # RNN 셀 구현
        self.fc = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층 구현

    def forward(self, x): # 구현한 RNN 셀과 출력층을 연결
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

클래스로 정의한 모델을 net에 저장한다.

In [12]:
net = Net(input_size, hidden_size, output_size)

이제 입력된 모델에 입력을 넣어서 출력의 크기를 확인해보자.

In [13]:
outputs = net(X)
print(outputs.shape)

torch.Size([1, 5, 5])


(1,5,5)의 크기를 갖는데, 각각  
배치 차원,  
시점,  
출력의 크기   
이다. 나중에 정확도를 측정할 때는 이를 모두 펼쳐서 계산하게 되는데,  
이때는 view를 사용하여 배치 차원과 시점 차원을 하나로 만든다. 

In [14]:
print(outputs.view(-1, input_size).shape) # 2차원 텐서로 변환

torch.Size([5, 5])


이제 레이블 데이터의 크기를 리마인드 해보자.

In [15]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([1, 5])
torch.Size([5])


레이블 데이터는 (1,5)의 크기를 가지는데,   
마찬가지로 나중에 정확도를 측정할 때는 이를 펼쳐서 계산할 예정이다.   
이 경우 (5)의 크기를 가지게 된다.  

이제 옵티마이저와 손실 함수를 정의한다. 

In [16]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(),learning_rate)

In [17]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    # view를 하는 이유는 Batch 차원 제거를 위함.
    loss = criterion(outputs.view(-1, input_size), Y.view(-1))
    loss.backward() # 기울기 계산
    optimizer.step() # 아까 optimizer 선언 시 넣어둔 파라미터 업데이트

    # 아래 세 줄은 모델이 실제 어떻게 예측했는지를 확인하는 코드.
    result = outputs.data.numpy().argmax(axis=2) # 최종 예측값인 각 time-step 별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택

    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])

    print(i, f'loss : {loss.item()}\nprediction : {result}\ntrue Y : {y_data}\n prediction str : {result_str}')

0 loss : 1.6507171392440796
prediction : [[2 2 2 2 2]]
true Y : [[4, 4, 3, 2, 0]]
 prediction str : eeeee
1 loss : 1.3870229721069336
prediction : [[2 4 4 2 2]]
true Y : [[4, 4, 3, 2, 0]]
 prediction str : eppee
2 loss : 1.1303677558898926
prediction : [[4 4 4 2 0]]
true Y : [[4, 4, 3, 2, 0]]
 prediction str : pppe!
3 loss : 0.8727928400039673
prediction : [[4 4 3 2 0]]
true Y : [[4, 4, 3, 2, 0]]
 prediction str : pple!
4 loss : 0.6350645422935486
prediction : [[4 4 3 2 0]]
true Y : [[4, 4, 3, 2, 0]]
 prediction str : pple!
5 loss : 0.4534829258918762
prediction : [[4 4 3 2 0]]
true Y : [[4, 4, 3, 2, 0]]
 prediction str : pple!
6 loss : 0.3227476179599762
prediction : [[4 4 3 2 0]]
true Y : [[4, 4, 3, 2, 0]]
 prediction str : pple!
7 loss : 0.22444100677967072
prediction : [[4 4 3 2 0]]
true Y : [[4, 4, 3, 2, 0]]
 prediction str : pple!
8 loss : 0.1529284566640854
prediction : [[4 4 3 2 0]]
true Y : [[4, 4, 3, 2, 0]]
 prediction str : pple!
9 loss : 0.10256731510162354
prediction : [[4

### 2. 더 많은 데이터로 학습한 문자 단위 RNN(Char RNN)

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim

#### 1) 훈련 데이터 전처리하기

In [19]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")


문자 집합을 생성하고, 각 문자에 고유한 정수를 부여한다.

In [20]:
char_set = list(set(sentence)) # 중복을 제거한 문자 집합 생성
char_dic = {c : i for i, c in enumerate(char_set)} # 각 문자에 정수 인코딩

In [21]:
print(char_dic)

{'g': 0, 's': 1, 'l': 2, 'y': 3, 'r': 4, 'i': 5, '.': 6, 'u': 7, 't': 8, 'e': 9, 'f': 10, 'a': 11, 'p': 12, ' ': 13, 'c': 14, "'": 15, 'b': 16, 'w': 17, 'd': 18, 'm': 19, ',': 20, 'h': 21, 'k': 22, 'o': 23, 'n': 24}


각 문자에 정수가 부여되었으며, 총 25개의 문자가 존재한다.  
문자 집합의 크기를 확인해보자.  

In [22]:
dic_size = len(char_dic)
print(f'문자 집합의 크기 : {dic_size}')

문자 집합의 크기 : 25


문자 집합의 크기는 25이며,  
입력은 원-핫 벡터로 사용할 것이므로 매 시점마다 들어갈 입력의 크기이기도 하다.

이제 하이퍼파라미터를 설정하자.  
hidden_size(은닉 상태의 크기)를 입력의 크기와 동일하게 줬는데,  
이는 사용자의 선택으로 다른 값을 줘도 무방하다.  

그리고 sequence_length라는 변수를 선언했는데,  
우리가 앞서 만든 샘플을 10개 단위로 끊어서 샘플을 만들 예정이기 때문이다.

In [23]:
# 하이퍼파라미터 설정
hidden_size = dic_size
sequence_length = 10   # 임의 숫자 지정
learning_rate = 0.1

다음은 임의로 지정한 sequence_length 값인 10의 단위로 샘플들을 잘라서 데이터를 만든다.

In [29]:
# 데이터 구성
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    # 슬라이싱 길이 계산 : (끝 인덱스 - 시작 인덱스)가 슬라이싱 길이이다. 
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)

    x_data.append([char_dic[c] for c in x_str])
    y_data.append([char_dic[c] for c in y_str])

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

#### range 횟수(슬라이싱 가능 횟수) 설정이 어떻게 저렇게 나왔을까?
- x_str,y_str에 마지막 인덱스(len(sentence) - sequence_length - 1)가 넘겨졌을 때를 보자.  
- x_str       
= (len(sentence) - sequence_length - 1) : (len(sentence) - sequence_length -1 + sequence_length)  
= len(sentence) - sequence_length - 1 : len(sentence) - 1

- y_str  
= (len(sentence) - sequence_length - 1 + 1): (len(sentence) - sequence_length - 1 + sequence_length + 1)  
= len(sentence) - sequence_length : len(sentence)

핵심은 y_str의 슬라이싱 범위의 마지막에 있다.  
(i + sequence_length + 1)이면,  
range의 마지막 인덱스(len(sentence) - sequence_length - 1)가 넘겨지면, len(sentence)가 된다.  

총 170개의 샘플이 생성되었다.  
그리고 각 샘플의 각 문자들은 고유한 정수로 인코딩이 된 상태이다.  
첫 번째 샘플의 입력 데이터와 레이블 데이터를 출력해보자.

In [30]:
print(x_data[0])
print(y_data[0])

[5, 10, 13, 3, 23, 7, 13, 17, 11, 24]
[10, 13, 3, 23, 7, 13, 17, 11, 24, 8]


입력 시퀀스에 대해 원-핫 인코딩을 수행하고, 입력 데이터와 레이블 데이터를 텐서로 변환한다.  

In [34]:
# np.eye()는 주어진 크기의 단위행렬(혹은 대각행렬) 생성
# 단위행렬은, 주대각선(왼쪽 상단에서 오른쪽 하단 방향) 원소가 1이고, 나머지는 모두 0인 행렬
# dic_size는 문자 집합의 크기

# x_data는 단위문자를 정수로 맵핑한 데이터였다.
# np.eye(dic_size)에 [x]를 붙이면, 단위행렬의 x번째 행을 선택하게 된다.
# 예컨대 np.eye(3)[0] -> [1.,0.,0.]이 되는것이다.
# 즉 맵핑된 정수의 인덱스에 '1'을 넣는 것과 같은 효과가 나온다.
# 따라서 원-핫 인코딩이 된다.
x_one_hot = [np.eye(dic_size)[x] for x in x_data] # x 데이터는 원-핫 인코딩
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [35]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([170, 10, 25])
레이블의 크기 : torch.Size([170, 10])


In [ ]:
print(X[0])
'''tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.], i
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.], f
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.], 공백
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.], y
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0.], o
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.], y
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.], 공백
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0.], w
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.], a
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1.]]) n '''

tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [37]:
print(Y[0])

tensor([10, 13,  3, 23,  7, 13, 17, 11, 24,  8])


#### 2) 모델 구현하기

앞서 실습한 문자 단위 RNN과 거의 동일하다.  
단 이번에는 은닉층을 두 개 쌓는다.  

In [38]:
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        # 현재 hidden_size는 dic_size와 같음.
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [39]:
net = Net(dic_size, hidden_size, 2) # 이번에는 층이 2개이다.

nn.RNN() 안에 num_layers라는 인자를 사용한다. 은닉층을 몇 개 쌓을 것인지를 설정한다.  
모델 선언 시 layers라는 인자에 2를 전달하여 은닉층을 두 개 쌓는다.  

이제 비용함수와 옵티마이저를 선언한다.

In [40]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

다음은 모델에 입력을 넣어서 출력의 크기를 확인해본다.  
각각, (배치 차원, 시점, 출력의 크기)이다.

In [41]:
outputs = net(X)
print(outputs.shape) # 3차원 텐서

torch.Size([170, 10, 25])


나중에 정확도를 측정할 때는 이를 모두 펼쳐서 계산한다.  
view를 사용하여 배치 차원과 시점 차원을 하나로 만든다.  

In [42]:
print(outputs.view(-1,dic_size).shape) # 2차원 텐서로 전환

torch.Size([1700, 25])


잠깐 레이블 데이터의 크기를 리마인드해보자.

In [43]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([170, 10])
torch.Size([1700])


In [44]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X) # (170, 10, 25) 크기를 가진 텐서를 매 에포트마다 모델의 입력으로 사용

    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    # results의 텐서 크기는 (170,10)
    results = outputs.argmax(dim=2)
    predict_str = ''
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오지만
            predict_str += ''.join([char_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += char_set[result[-1]]

    print(predict_str)

ha    a           a     a a'             a               a                   aa          a           a   aa               a      a    h           a      aa        aa              
                                                                                                                                                                                   
t''.dt'',dhthddhdht dhdhdlddthdthed'dhhthddhddhddhhdlddt'dthdkhdmddht.edtt,edmp dttedbdht'dhhedhdhddhtt,dtt dttedhdlt'dht'dhht,hephddhdhddddtt.ddht ddhedlddhtt,dhthdtddthdphddhedc
t lc  ia     e e      e            o    o           e        e       e  e        e            o              l       o        e       l          e       o                        o
 et tr tr  th st   t  e it  t  tett ehst  ktet tt  t       heh ih thh ht    sttse sh  she   tt ei st st eh  s th  ehe s e ittst it t h  st st ehh t e sh  thst h  s  et   o  i  tt 
 tt t so tttototo totototodototosototodt no tooo tototheoto totodotoo otost totototototodot otoht to